In [1]:
# import the libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD
from keras import regularizers
import csv
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [14]:
temperatures = [] # emptz list for the temperatures
count = 0
with open('bud_temp.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row[0] != 'date': # do not read the first row
            temperatures.append((float(row[1])+float(row[2]))/2) # load the average temeratures for each daz since 2012
            count += 1
print('Loaded',count,'temperature data') # data count
temperatures = np.asarray(temperatures)
temp_avg = np.average(temperatures)
temp_mean = np.mean(temperatures)
temperatures = (temperatures - temp_avg) / temp_mean # normalize the data for our purposes

Loaded 3218 temperature data


In [3]:
x_data = [] # empty arrays for the input and output data
y1_data = []
y2_data = []
y3_data = []
for i in range(count-365-28-1):
    x_data.append(temperatures[i:i+365]) # the input of the neural network is the average temperature of the last 365 days
    y1_data.append(temperatures[i+365+1]) # the output of the first network is the average temperature of the very next day after the last input
    y2_data.append(temperatures[i+365+7]) # the output of the second network is the average temperature of the day after 7 days from the last input
    y3_data.append(temperatures[i+365+28]) # the output of the third network is the average temperature of the day after 28 days from the last input
x_data = np.asarray(x_data)
y1_data = np.asarray(y1_data)
y2_data = np.asarray(y2_data)
y3_data = np.asarray(y3_data)

In [4]:
model = Sequential() # create the model
model.add(Dense(100, input_shape=(x_data.shape[1],))) # the first hidden layer is a 100 neuron flat layer with sigmoid activation
model.add(Activation('sigmoid'))
model.add(Dense(10)) # the first hidden layer is a 10 neuron flat layer with sigmoid activation
model.add(Activation('sigmoid'))
model.add(Dense(1))# the output layer is a single nouron with tanh activation
model.add(Activation('tanh'))
model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.1, momentum=0.001, nesterov=True)) # compile the network
early_stopping = EarlyStopping(patience=10, monitor='val_loss') # create the early stopping function

In [5]:
# split the data to train and test arrays for each model
x1_train, x1_test, y1_train, y1_test = train_test_split(x_data, y1_data, test_size=0.2, shuffle=True)
x2_train, x2_test, y2_train, y2_test = train_test_split(x_data, y2_data, test_size=0.2, shuffle=True)
x3_train, x3_test, y3_train, y3_test = train_test_split(x_data, y3_data, test_size=0.2, shuffle=True)

In [6]:
# train the first model (prediction for the next day)
model.fit(x1_train, y1_train,
          batch_size=50,
          epochs=500,
          validation_data=(x1_test, y1_test),
          callbacks=[early_stopping],
          shuffle=True)

Train on 2259 samples, validate on 565 samples
Epoch 1/500
2259/2259 [==============================] - 1s 376us/sample - loss: 0.1585 - val_loss: 0.0975
Epoch 2/500
2259/2259 [==============================] - 0s 61us/sample - loss: 0.0993 - val_loss: 0.0944
Epoch 3/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0964 - val_loss: 0.0900
Epoch 4/500
2259/2259 [==============================] - 0s 62us/sample - loss: 0.0932 - val_loss: 0.0880
Epoch 5/500
2259/2259 [==============================] - 0s 57us/sample - loss: 0.0903 - val_loss: 0.0855
Epoch 6/500
2259/2259 [==============================] - 0s 57us/sample - loss: 0.0878 - val_loss: 0.0859
Epoch 7/500
2259/2259 [==============================] - 0s 59us/sample - loss: 0.0856 - val_loss: 0.0829
Epoch 8/500
2259/2259 [==============================] - 0s 56us/sample - loss: 0.0833 - val_loss: 0.0811
Epoch 9/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0814 - val_loss: 0.078

In [7]:
x_input = np.empty([1,365])
x_input[0] = temperatures[-365:] #the input data for the temperature predictions is the average temperature of the last 365 days

In [8]:
print('Temperature prediction for oct 28.:')
print(model.predict(x_input)[0][0]*temp_mean+temp_avg) # predict the output and the normalized data back to a celsius value

11.659762803262957


In [9]:
# train the first model (prediction for 7 days later)
model.fit(x2_train, y2_train,
          batch_size=50,
          epochs=500,
          validation_data=(x2_test, y2_test),
          callbacks=[early_stopping],
          shuffle=True)

Train on 2259 samples, validate on 565 samples
Epoch 1/500
2259/2259 [==============================] - 0s 64us/sample - loss: 0.0884 - val_loss: 0.0835
Epoch 2/500
2259/2259 [==============================] - 0s 57us/sample - loss: 0.0847 - val_loss: 0.0923
Epoch 3/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0828 - val_loss: 0.0776
Epoch 4/500
2259/2259 [==============================] - 0s 56us/sample - loss: 0.0808 - val_loss: 0.0743
Epoch 5/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0802 - val_loss: 0.0769
Epoch 6/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0786 - val_loss: 0.0732
Epoch 7/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0777 - val_loss: 0.0851
Epoch 8/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0776 - val_loss: 0.0735
Epoch 9/500
2259/2259 [==============================] - 0s 61us/sample - loss: 0.0774 - val_loss: 0.0732

In [10]:
print('Temperature prediction for nov. 3.:')
print(model.predict(x_input)[0][0]*temp_mean+temp_avg) # predict the output and the normalized data back to a celsius value

8.220788632524059


In [11]:
# train the first model (prediction for 28 days later)
model.fit(x3_train, y3_train,
          batch_size=50,
          epochs=500,
          validation_data=(x3_test, y3_test),
          callbacks=[early_stopping],
          shuffle=True)

Train on 2259 samples, validate on 565 samples
Epoch 1/500
2259/2259 [==============================] - 0s 63us/sample - loss: 0.0980 - val_loss: 0.1017
Epoch 2/500
2259/2259 [==============================] - 0s 61us/sample - loss: 0.0903 - val_loss: 0.1119
Epoch 3/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0862 - val_loss: 0.1010
Epoch 4/500
2259/2259 [==============================] - 0s 58us/sample - loss: 0.0852 - val_loss: 0.0975
Epoch 5/500
2259/2259 [==============================] - 0s 61us/sample - loss: 0.0837 - val_loss: 0.0940
Epoch 6/500
2259/2259 [==============================] - 0s 60us/sample - loss: 0.0819 - val_loss: 0.0910
Epoch 7/500
2259/2259 [==============================] - 0s 58us/sample - loss: 0.0819 - val_loss: 0.0897
Epoch 8/500
2259/2259 [==============================] - 0s 61us/sample - loss: 0.0811 - val_loss: 0.0938
Epoch 9/500
2259/2259 [==============================] - 0s 61us/sample - loss: 0.0801 - val_loss: 0.0959

In [13]:
print('Temperature prediction for nov. 24.:')
print(model.predict(x_input)[0][0]*temp_mean+temp_avg) # predict the output and the normalized data back to a celsius value

Temperature prediction for nov. 24.:
7.169753802539288
